# Phân tích Mạng Xã Hội K-pop

Notebook này thực hiện các thuật toán phân tích mạng xã hội:

1. **Chứng minh khái niệm thế giới nhỏ (Small World)**: Tính Average Shortest Path Length và Clustering Coefficient
2. **Xếp hạng các node bằng PageRank**: Xác định các node quan trọng nhất trong mạng
3. **Phát hiện cộng đồng (Community Detection)**: Tìm các cộng đồng trong mạng K-pop

## 1. Import thư viện

In [1]:
import json
import sys
import io
import math
import random
import os
from pathlib import Path
from typing import Dict, List, Any, Tuple
from collections import defaultdict

# Đổi working directory về root của project (nếu đang chạy từ notebooks/)
current_dir = Path.cwd()
if current_dir.name == 'notebooks':
    os.chdir(current_dir.parent)
    print(f"✓ Đã đổi working directory về: {os.getcwd()}")

# Robust UTF-8 console output on Windows
if sys.platform == 'win32':
    try:
        sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8', errors='replace')
        sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8', errors='replace')
    except Exception:
        pass

try:
    import networkx as nx
    print("✓ Import thư viện thành công")
    print(f"NetworkX version: {nx.__version__}")
except ImportError:
    print("❌ NetworkX chưa được cài đặt. Chạy: pip install networkx")
    raise

try:
    import matplotlib.pyplot as plt
    import matplotlib
    matplotlib.use('Agg')  # Non-interactive backend
    print("✓ Matplotlib đã sẵn sàng")
except ImportError:
    print("⚠️  Matplotlib chưa được cài đặt. Chạy: pip install matplotlib")

✓ Đã đổi working directory về: c:\Users\DELL\Social-network-analyst
✓ Import thư viện thành công
NetworkX version: 3.3
✓ Matplotlib đã sẵn sàng


## 2. Load dữ liệu

In [2]:
def load_graph_data(
    merged_file: str = "data/korean_artists_graph_bfs.json"
) -> Tuple[Dict, List]:
    """Load dữ liệu từ file merged"""
    # Tự động tìm đúng đường dẫn nếu file không tồn tại ở vị trí hiện tại
    if not os.path.exists(merged_file):
        # Thử tìm từ root của project
        root_dir = Path.cwd()
        if root_dir.name == 'notebooks':
            root_dir = root_dir.parent
        alt_path = root_dir / merged_file
        if alt_path.exists():
            merged_file = str(alt_path)
        else:
            # Thử đường dẫn tương đối từ notebooks/
            alt_path = Path('..') / merged_file
            if alt_path.exists():
                merged_file = str(alt_path.resolve())
    
    try:
        with open(merged_file, 'r', encoding='utf-8') as f:
            merged_data = json.load(f)
        
        nodes = merged_data.get("nodes", {})
        edges = merged_data.get("edges", [])
        metadata = merged_data.get("metadata", {})
        
        print(f"✓ Đã load từ {merged_file}")
        print(f"  - Nodes: {len(nodes)}")
        print(f"  - Edges: {len(edges)}")
        
        if metadata:
            print(f"\n📊 THỐNG KÊ TỪ METADATA:")
            if "total_nodes" in metadata:
                print(f"  - Tổng nodes: {metadata['total_nodes']}")
            if "total_edges" in metadata:
                print(f"  - Tổng edges: {metadata['total_edges']}")
            if "nodes_by_type" in metadata:
                print(f"\n  📋 Nodes theo loại:")
                for node_type, count in sorted(metadata["nodes_by_type"].items(), key=lambda x: -x[1]):
                    print(f"    - {node_type}: {count}")
            if "edges_by_type" in metadata:
                print(f"\n  📋 Edges theo loại:")
                for edge_type, count in sorted(metadata["edges_by_type"].items(), key=lambda x: -x[1]):
                    print(f"    - {edge_type}: {count}")
        
        print(f"\n📊 Tổng cộng: {len(nodes)} nodes, {len(edges)} edges")
        return nodes, edges
        
    except FileNotFoundError:
        print(f"❌ Không tìm thấy {merged_file}")
        print("   Vui lòng chạy merge_and_import_neo4j.py để tạo file merged trước")
        raise
    except Exception as e:
        print(f"❌ Lỗi khi load file: {e}")
        raise

# Load dữ liệu
nodes, edges = load_graph_data()

✓ Đã load từ data/korean_artists_graph_bfs.json
  - Nodes: 1692
  - Edges: 3932

📊 THỐNG KÊ TỪ METADATA:

📊 Tổng cộng: 1692 nodes, 3932 edges


## 3. Xây dựng NetworkX Graph

In [3]:
def build_networkx_graph(nodes: Dict, edges: List, undirected: bool = True) -> nx.Graph:
    """Xây dựng NetworkX graph từ nodes và edges"""
    if undirected:
        G = nx.Graph()
    else:
        G = nx.DiGraph()
    
    # Thêm nodes
    for node_id, node_data in nodes.items():
        G.add_node(node_id, **{
            'label': node_data.get('label', 'Entity'),
            'title': node_data.get('title', node_id)
        })
    
    # Thêm edges
    for edge in edges:
        src = edge.get('source')
        tgt = edge.get('target')
        if src and tgt and src in nodes and tgt in nodes:
            G.add_edge(src, tgt, type=edge.get('type', 'RELATED_TO'))
    
    return G

# Build graph
G = build_networkx_graph(nodes, edges, undirected=True)
print(f"✓ Graph có {G.number_of_nodes()} nodes và {G.number_of_edges()} edges")

# Thống kê nodes theo label
label_counts = defaultdict(int)
for node_id in G.nodes():
    label = G.nodes[node_id].get('label', 'Unknown')
    label_counts[label] += 1

print(f"\n📊 THỐNG KÊ NODES THEO LABEL:")
for label, count in sorted(label_counts.items(), key=lambda x: -x[1]):
    percentage = 100 * count / G.number_of_nodes()
    print(f"  {label}: {count} ({percentage:.1f}%)")

✓ Graph có 1692 nodes và 3894 edges

📊 THỐNG KÊ NODES THEO LABEL:
  Artist: 452 (26.7%)
  Song: 262 (15.5%)
  Album: 258 (15.2%)
  Company: 240 (14.2%)
  Group: 213 (12.6%)
  Genre: 175 (10.3%)
  Occupation: 60 (3.5%)
  Instrument: 32 (1.9%)


## 4. Phân tích Khái niệm Thế giới Nhỏ (Small World)

In [4]:
def analyze_small_world(G: 'nx.Graph') -> Tuple[Dict[str, Any], 'nx.Graph']:
    """
    Phân tích khái niệm Small World:
    - Tính Average Shortest Path Length (APL)
    - Tính Clustering Coefficient
    - So sánh với random graph cùng kích thước
    """
    print("\n" + "=" * 70)
    print("1. PHÂN TÍCH KHÁI NIỆM THẾ GIỚI NHỎ (SMALL WORLD)")
    print("=" * 70)
    
    results = {
        "total_nodes": G.number_of_nodes(),
        "total_edges": G.number_of_edges(),
    }
    
    # Kiểm tra connected components
    if nx.is_connected(G):
        components = [G]
        largest_cc = G
        print(f"\n📊 Graph là connected với {G.number_of_nodes()} nodes")
    else:
        components = list(nx.connected_components(G))
        largest_cc = G.subgraph(max(components, key=len)).copy()
        print(f"\n📊 Graph có {len(components)} connected components")
        print(f"   Largest component: {largest_cc.number_of_nodes()} nodes ({100*largest_cc.number_of_nodes()/G.number_of_nodes():.1f}%)")
    
    results["num_components"] = len(components) if not nx.is_connected(G) else 1
    results["largest_component_size"] = largest_cc.number_of_nodes()
    results["largest_component_percentage"] = 100 * largest_cc.number_of_nodes() / G.number_of_nodes()
    
    # Tính Average Shortest Path Length trên largest component
    print("\n🔍 Tính Average Shortest Path Length (APL)...")
    
    n = largest_cc.number_of_nodes()
    m = largest_cc.number_of_edges()
    
    if n > 5000:
        # Sampling cho graph lớn
        print(f"   Graph lớn ({n} nodes), sử dụng sampling...")
        sample_size = min(1000, n)
        sample_nodes = random.sample(list(largest_cc.nodes()), sample_size)
        
        total_paths = 0
        path_count = 0
        
        for i, source in enumerate(sample_nodes):
            if i % 100 == 0:
                print(f"   Đang xử lý node {i}/{sample_size}...")
            lengths = nx.single_source_shortest_path_length(largest_cc, source)
            for target, length in lengths.items():
                if source != target:
                    total_paths += length
                    path_count += 1
        
        if path_count > 0:
            apl = total_paths / path_count
        else:
            apl = float('inf')
        print(f"   (Ước lượng từ {sample_size} nodes)")
    else:
        apl = nx.average_shortest_path_length(largest_cc)
    
    results["average_path_length"] = apl
    print(f"   ✓ Average Shortest Path Length: {apl:.4f}")
    
    # Tính Clustering Coefficient
    print("\n🔍 Tính Clustering Coefficient...")
    avg_clustering = nx.average_clustering(G)
    results["clustering_coefficient"] = avg_clustering
    print(f"   ✓ Average Clustering Coefficient: {avg_clustering:.4f}")
    
    # Tính diameter (đường kính)
    print("\n🔍 Tính Diameter...")
    if n <= 5000:
        diameter = nx.diameter(largest_cc)
    else:
        # Ước lượng diameter bằng sampling
        sample_nodes = random.sample(list(largest_cc.nodes()), min(100, n))
        max_dist = 0
        for node in sample_nodes:
            eccentricity = nx.eccentricity(largest_cc, v=node)
            max_dist = max(max_dist, eccentricity)
        diameter = max_dist
        print(f"   (Ước lượng từ sampling)")
    results["diameter"] = diameter
    print(f"   ✓ Diameter: {diameter}")
    
    return results, largest_cc

# Chạy phân tích Small World
small_world_results, largest_cc = analyze_small_world(G)


1. PHÂN TÍCH KHÁI NIỆM THẾ GIỚI NHỎ (SMALL WORLD)

📊 Graph có 158 connected components
   Largest component: 1521 nodes (89.9%)

🔍 Tính Average Shortest Path Length (APL)...
   ✓ Average Shortest Path Length: 3.9393

🔍 Tính Clustering Coefficient...
   ✓ Average Clustering Coefficient: 0.1124

🔍 Tính Diameter...
   ✓ Diameter: 9


In [5]:
# So sánh với Random Graph (Erdős–Rényi)
print("\n🔍 So sánh với Random Graph (Erdős–Rényi)...")

n = largest_cc.number_of_nodes()
m = largest_cc.number_of_edges()
p = 2 * m / (n * (n - 1)) if n > 1 else 0  # Probability để có cùng số edges

# Lý thuyết cho random graph:
avg_degree = 2 * m / n if n > 0 else 0

if avg_degree > 1:
    expected_apl_random = math.log(n) / math.log(avg_degree) if avg_degree > 1 else float('inf')
else:
    expected_apl_random = float('inf')
expected_clustering_random = avg_degree / n if n > 0 else 0

small_world_results["random_graph_expected_apl"] = expected_apl_random
small_world_results["random_graph_expected_clustering"] = expected_clustering_random
small_world_results["average_degree"] = avg_degree

print(f"   Average Degree: {avg_degree:.2f}")
print(f"   Random Graph Expected APL: {expected_apl_random:.4f}")
print(f"   Random Graph Expected Clustering: {expected_clustering_random:.6f}")

# Small World Index
# σ = (C/C_random) / (L/L_random)
# σ > 1 indicates small world property
apl = small_world_results["average_path_length"]
avg_clustering = small_world_results["clustering_coefficient"]

if expected_clustering_random > 0 and expected_apl_random > 0 and expected_apl_random != float('inf'):
    sigma = (avg_clustering / expected_clustering_random) / (apl / expected_apl_random)
    small_world_results["small_world_sigma"] = sigma
    print(f"\n📊 Small World Sigma (σ): {sigma:.4f}")
    
    if sigma > 1:
        print("   ✓ σ > 1: Mạng có tính chất THẾ GIỚI NHỎ (Small World)")
    else:
        print("   ✗ σ ≤ 1: Mạng không thể hiện tính chất Small World rõ ràng")
else:
    small_world_results["small_world_sigma"] = None

# Kết luận
print("\n" + "-" * 70)
print("📋 KẾT LUẬN VỀ KHÁI NIỆM THẾ GIỚI NHỎ:")
print("-" * 70)

conclusions = []

# APL thấp?
if apl < math.log(n) * 2:
    conclusions.append(f"✓ APL = {apl:.2f} khá thấp (< 2*ln(n) = {math.log(n)*2:.2f})")
    small_world_results["low_apl"] = True
else:
    conclusions.append(f"✗ APL = {apl:.2f} khá cao")
    small_world_results["low_apl"] = False

# Clustering cao?
if avg_clustering > expected_clustering_random * 10:
    conclusions.append(f"✓ Clustering = {avg_clustering:.4f} cao hơn random {avg_clustering/expected_clustering_random:.1f}x")
    small_world_results["high_clustering"] = True
else:
    conclusions.append(f"✗ Clustering = {avg_clustering:.4f} không cao hơn random đáng kể")
    small_world_results["high_clustering"] = False

# Six Degrees of Separation?
if apl <= 6:
    conclusions.append(f"✓ APL ≤ 6: Tuân theo 'Six Degrees of Separation'")
    small_world_results["six_degrees"] = True
else:
    conclusions.append(f"✗ APL > 6: Không tuân theo 'Six Degrees of Separation' nghiêm ngặt")
    small_world_results["six_degrees"] = False

for c in conclusions:
    print(f"   {c}")

if small_world_results.get("low_apl") and small_world_results.get("high_clustering"):
    print("\n🎯 KẾT LUẬN: Mạng K-pop THỎA MÃN tính chất THẾ GIỚI NHỎ (Small World)")
    print(f"   - Bất kỳ 2 node nào cũng có thể kết nối qua trung bình {apl:.1f} bước")
    print(f"   - Các node có xu hướng tạo thành các cụm (cluster) cục bộ")
    small_world_results["is_small_world"] = True
else:
    print("\n🎯 KẾT LUẬN: Mạng có một số đặc điểm của Small World nhưng chưa hoàn toàn")
    small_world_results["is_small_world"] = False


🔍 So sánh với Random Graph (Erdős–Rényi)...
   Average Degree: 5.10
   Random Graph Expected APL: 4.4962
   Random Graph Expected Clustering: 0.003354

📊 Small World Sigma (σ): 38.2494
   ✓ σ > 1: Mạng có tính chất THẾ GIỚI NHỎ (Small World)

----------------------------------------------------------------------
📋 KẾT LUẬN VỀ KHÁI NIỆM THẾ GIỚI NHỎ:
----------------------------------------------------------------------
   ✓ APL = 3.94 khá thấp (< 2*ln(n) = 14.65)
   ✓ Clustering = 0.1124 cao hơn random 33.5x
   ✓ APL ≤ 6: Tuân theo 'Six Degrees of Separation'

🎯 KẾT LUẬN: Mạng K-pop THỎA MÃN tính chất THẾ GIỚI NHỎ (Small World)
   - Bất kỳ 2 node nào cũng có thể kết nối qua trung bình 3.9 bước
   - Các node có xu hướng tạo thành các cụm (cluster) cục bộ


In [6]:
# Vẽ biểu đồ phân bố
try:
    import matplotlib.pyplot as plt
    MATPLOTLIB_AVAILABLE = True
except ImportError:
    MATPLOTLIB_AVAILABLE = False

if MATPLOTLIB_AVAILABLE:
    print("\n📊 Vẽ biểu đồ phân bố...")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Biểu đồ phân bố degree
    degrees = [G.degree(n) for n in G.nodes()]
    ax1.hist(degrees, bins=50, edgecolor='black', alpha=0.7)
    ax1.set_xlabel('Degree')
    ax1.set_ylabel('Số lượng nodes')
    ax1.set_title('Phân bố Degree')
    ax1.grid(True, alpha=0.3)
    
    # Biểu đồ phân bố shortest path length
    if n <= 5000:
        path_lengths = []
        for source in list(largest_cc.nodes())[:100]:  # Sample để nhanh hơn
            lengths = nx.single_source_shortest_path_length(largest_cc, source)
            for target, length in lengths.items():
                if source != target:
                    path_lengths.append(length)
        
        ax2.hist(path_lengths, bins=20, edgecolor='black', alpha=0.7, color='orange')
        ax2.set_xlabel('Shortest Path Length')
        ax2.set_ylabel('Số lượng cặp nodes')
        ax2.set_title('Phân bố Shortest Path Length')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('small_world_analysis.png', dpi=150, bbox_inches='tight')
    print("✓ Đã lưu biểu đồ vào small_world_analysis.png")
    plt.show()


📊 Vẽ biểu đồ phân bố...
✓ Đã lưu biểu đồ vào small_world_analysis.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_1572\3164417652.py:39: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 5. Xếp hạng Node bằng PageRank

In [7]:
def analyze_pagerank(G: 'nx.Graph', top_k: int = 50) -> Tuple[Dict[str, Any], Dict, List]:
    """
    Xếp hạng nodes bằng thuật toán PageRank
    """
    print("\n" + "=" * 70)
    print("2. XẾP HẠNG NODE BẰNG PAGERANK")
    print("=" * 70)
    
    results = {}
    
    # Tính PageRank
    print("\n🔍 Đang tính PageRank...")
    pagerank = nx.pagerank(G, alpha=0.85, max_iter=100)
    
    # Sắp xếp theo PageRank giảm dần
    sorted_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)
    
    results["total_nodes"] = len(pagerank)
    results["top_nodes"] = []
    
    print(f"\n📊 TOP {top_k} NODES THEO PAGERANK:")
    print("-" * 70)
    print(f"{'Rank':<6} {'Node':<40} {'PageRank':<12} {'Label'}")
    print("-" * 70)
    
    for i, (node, score) in enumerate(sorted_pagerank[:top_k], 1):
        label = G.nodes[node].get('label', 'Unknown')
        print(f"{i:<6} {node[:38]:<40} {score:.8f}   {label}")
        results["top_nodes"].append({
            "rank": i,
            "node": node,
            "pagerank": score,
            "label": label
        })
    
    return results, pagerank, sorted_pagerank

# Chạy phân tích PageRank
pagerank_results, pagerank, sorted_pagerank = analyze_pagerank(G, top_k=30)


2. XẾP HẠNG NODE BẰNG PAGERANK

🔍 Đang tính PageRank...

📊 TOP 30 NODES THEO PAGERANK:
----------------------------------------------------------------------
Rank   Node                                     PageRank     Label
----------------------------------------------------------------------
1      Occupation_Diễn viên                     0.02398023   Occupation
2      Genre_R&B                                0.02000889   Genre
3      Genre_Dance-pop                          0.01266154   Genre
4      Genre_Hip hop                            0.01205530   Genre
5      Occupation_Nhạc sĩ                       0.01049384   Occupation
6      Genre_Pop                                0.00997567   Genre
7      BTS                                      0.00796211   Group
8      Genre_Ballad                             0.00773723   Genre
9      Girls' Generation                        0.00724393   Group
10     Occupation_Rapper                        0.00695195   Occupation
11     GFriend    

In [8]:
# Thống kê theo label
print("\n📊 PAGERANK TRUNG BÌNH THEO LABEL:")
print("-" * 50)

label_scores = defaultdict(list)
for node, score in pagerank.items():
    label = G.nodes[node].get('label', 'Unknown')
    label_scores[label].append(score)

label_avg = {}
for label, scores in label_scores.items():
    avg = sum(scores) / len(scores)
    label_avg[label] = avg

pagerank_results["pagerank_by_label"] = {}
for label, avg in sorted(label_avg.items(), key=lambda x: x[1], reverse=True):
    count = len(label_scores[label])
    print(f"  {label:<15}: {avg:.8f} (n={count})")
    pagerank_results["pagerank_by_label"][label] = {
        "average": avg,
        "count": count
    }

# Tìm top node theo từng label
top_by_label = {}
for node, score in sorted_pagerank:
    label = G.nodes[node].get('label', 'Unknown')
    if label not in top_by_label:
        top_by_label[label] = (node, score)

print("\n   🏆 Top node theo từng loại:")
for label, (node, score) in sorted(top_by_label.items(), key=lambda x: x[1][1], reverse=True):
    print(f"      - {label}: {node} ({score:.6f})")

pagerank_results["top_by_label"] = {label: {"node": node, "pagerank": score} for label, (node, score) in top_by_label.items()}


📊 PAGERANK TRUNG BÌNH THEO LABEL:
--------------------------------------------------
  Occupation     : 0.00126537 (n=60)
  Group          : 0.00090990 (n=213)
  Genre          : 0.00070417 (n=175)
  Artist         : 0.00068600 (n=452)
  Instrument     : 0.00053083 (n=32)
  Company        : 0.00047249 (n=240)
  Song           : 0.00034363 (n=262)
  Album          : 0.00029672 (n=258)

   🏆 Top node theo từng loại:
      - Occupation: Occupation_Diễn viên (0.023980)
      - Genre: Genre_R&B (0.020009)
      - Group: BTS (0.007962)
      - Company: Company_JYP Entertainment (0.006720)
      - Instrument: Instrument_Piano (0.004587)
      - Artist: IU (ca sĩ) (0.004489)
      - Album: Absolute First Album (0.001018)
      - Song: Blood Sweat & Tears (bài hát) (0.000711)


In [9]:
# Vẽ biểu đồ PageRank (SỬA LỖI KeyError)
if MATPLOTLIB_AVAILABLE:
    try:
        import pandas as pd
        
        # Tạo DataFrame cho top nodes
        top_df = pd.DataFrame(pagerank_results["top_nodes"][:30])
        
        fig, ax = plt.subplots(figsize=(12, 8))
        # SỬA: Dùng 'pagerank' thay vì 'PageRank'
        ax.barh(range(len(top_df)), top_df['pagerank'], color='steelblue')
        ax.set_yticks(range(len(top_df)))
        # SỬA: Dùng 'rank', 'node' thay vì 'Rank', 'Node'
        ax.set_yticklabels([f"{row['rank']}. {row['node'][:40]}" for _, row in top_df.iterrows()], fontsize=9)
        ax.set_xlabel('PageRank Score')
        ax.set_title('TOP 30 NODES THEO PAGERANK')
        ax.invert_yaxis()
        plt.tight_layout()
        plt.savefig('outputs/pagerank_analysis.png', dpi=150, bbox_inches='tight')
        print("✓ Đã lưu biểu đồ vào outputs/pagerank_analysis.png")
        plt.show()
        
        # Hiển thị bảng
        # SỬA: Dùng 'rank', 'node', 'label', 'pagerank' thay vì 'Rank', 'Node', 'Label', 'PageRank'
        display(top_df[['rank', 'node', 'label', 'pagerank']].head(30))
    except ImportError:
        print("⚠️  Pandas chưa được cài đặt, bỏ qua visualization")
    except Exception as e:
        print(f"⚠️  Lỗi: {e}")


✓ Đã lưu biểu đồ vào outputs/pagerank_analysis.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_1572\3438773613.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


,rank,node,label,pagerank
0,1,Occupation_Diễn viên,Occupation,0.023980
1,2,Genre_R&B,Genre,0.020009
2,3,Genre_Dance-pop,Genre,0.012662
3,4,Genre_Hip hop,Genre,0.012055
4,5,Occupation_Nhạc sĩ,Occupation,0.010494
5,6,Genre_Pop,Genre,0.009976
6,7,BTS,Group,0.007962
7,8,Genre_Ballad,Genre,0.007737
8,9,Girls' Generation,Group,0.007244
9,10,Occupation_Rapper,Occupation,0.006952


In [10]:
# Phân tích PageRank theo từng loại node (Top 5 mỗi loại)
print("\n" + "=" * 70)
print("📊 PAGERANK THEO TỪNG LOẠI NODE (TOP 5 MỖI LOẠI)")
print("=" * 70)

# Nhóm nodes theo label
nodes_by_label = defaultdict(list)
for node, score in sorted_pagerank:
    label = G.nodes[node].get('label', 'Unknown')
    nodes_by_label[label].append((node, score))

# Sắp xếp các label theo số lượng nodes
sorted_labels = sorted(nodes_by_label.items(), key=lambda x: len(x[1]), reverse=True)

# Hiển thị top 5 nodes cho mỗi loại
pagerank_by_type = {}
for label, nodes_list in sorted_labels:
    top_5 = nodes_list[:5]
    pagerank_by_type[label] = top_5
    
    print(f"\n🏷️  {label} (Tổng: {len(nodes_list)} nodes)")
    print("-" * 70)
    print(f"{'Rank':<6} {'Node':<45} {'PageRank':<15}")
    print("-" * 70)
    
    for i, (node, score) in enumerate(top_5, 1):
        print(f"{i:<6} {node[:43]:<45} {score:.8f}")

# Lưu vào results
pagerank_results["top_by_type"] = {
    label: [{"node": node, "pagerank": score} for node, score in top_5]
    for label, top_5 in pagerank_by_type.items()
}

# Vẽ biểu đồ PageRank theo từng loại (nếu có matplotlib)
if MATPLOTLIB_AVAILABLE:
    try:
        import pandas as pd
        
        # Tạo subplot cho mỗi loại có nhiều nodes nhất
        top_labels = sorted_labels[:6]  # Top 6 loại có nhiều nodes nhất
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        for idx, (label, nodes_list) in enumerate(top_labels):
            if idx >= 6:
                break
                
            top_5 = nodes_list[:5]
            nodes_names = [n[0][:30] for n in top_5]
            scores = [n[1] for n in top_5]
            
            ax = axes[idx]
            ax.barh(range(len(nodes_names)), scores, color='steelblue', alpha=0.7)
            ax.set_yticks(range(len(nodes_names)))
            ax.set_yticklabels(nodes_names, fontsize=8)
            ax.set_xlabel('PageRank Score', fontsize=9)
            ax.set_title(f'{label}\n(Top 5/{len(nodes_list)})', fontsize=10, fontweight='bold')
            ax.invert_yaxis()
            ax.grid(True, alpha=0.3, axis='x')
        
        # Ẩn các subplot không dùng
        for idx in range(len(top_labels), 6):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig('outputs/pagerank_by_type.png', dpi=150, bbox_inches='tight')
        print("\n✓ Đã lưu biểu đồ PageRank theo loại vào outputs/pagerank_by_type.png")
        plt.show()
        
    except Exception as e:
        print(f"\n⚠️  Lỗi khi vẽ biểu đồ: {e}")



📊 PAGERANK THEO TỪNG LOẠI NODE (TOP 5 MỖI LOẠI)

🏷️  Artist (Tổng: 452 nodes)
----------------------------------------------------------------------
Rank   Node                                          PageRank       
----------------------------------------------------------------------
1      IU (ca sĩ)                                    0.00448921
2      Yoo Young-jin                                 0.00422860
3      G-Dragon                                      0.00403882
4      RM (rapper)                                   0.00347527
5      Kenzie                                        0.00328834

🏷️  Song (Tổng: 262 nodes)
----------------------------------------------------------------------
Rank   Node                                          PageRank       
----------------------------------------------------------------------
1      Blood Sweat & Tears (bài hát)                 0.00071134
2      Loser (bài hát của Big Bang)                  0.00070149
3      Lovesick Girls  


✓ Đã lưu biểu đồ PageRank theo loại vào outputs/pagerank_by_type.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_1572\1681500439.py:70: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 6. Phát hiện Cộng đồng (Community Detection)


In [11]:
def analyze_communities(G: 'nx.Graph', top_k_communities: int = 10) -> Tuple[Dict[str, Any], List]:
    """
    Phát hiện cộng đồng trong mạng sử dụng thuật toán Louvain
    """
    print("\n" + "=" * 70)
    print("3. PHÁT HIỆN CỘNG ĐỒNG (COMMUNITY DETECTION)")
    print("=" * 70)
    
    results = {}
    
    # Kiểm tra có thư viện community detection không
    try:
        from networkx.algorithms import community as nx_community
        HAS_LOUVAIN = hasattr(nx_community, 'louvain_communities')
    except ImportError:
        HAS_LOUVAIN = False
    
    if HAS_LOUVAIN:
        print("\n🔍 Sử dụng thuật toán Louvain...")
        communities = nx_community.louvain_communities(G, seed=42)
        method = "Louvain"
    else:
        print("\n🔍 Sử dụng thuật toán Greedy Modularity...")
        communities = list(nx_community.greedy_modularity_communities(G))
        method = "Greedy Modularity"
    
    # Chuyển thành list để sắp xếp
    communities = [set(c) for c in communities]
    communities.sort(key=len, reverse=True)
    
    results["method"] = method
    results["total_communities"] = len(communities)
    
    print(f"\n✓ Phát hiện được {len(communities)} cộng đồng")
    
    # Tính modularity
    try:
        modularity = nx_community.modularity(G, communities)
        results["modularity"] = modularity
        print(f"✓ Modularity: {modularity:.4f}")
        
        if modularity > 0.3:
            print("   → Modularity > 0.3: Cấu trúc cộng đồng RÕ RÀNG")
        elif modularity > 0.1:
            print("   → Modularity > 0.1: Cấu trúc cộng đồng TRUNG BÌNH")
        else:
            print("   → Modularity ≤ 0.1: Cấu trúc cộng đồng YẾU")
    except:
        results["modularity"] = None
    
    # Thống kê kích thước cộng đồng
    community_sizes = [len(c) for c in communities]
    results["community_sizes"] = {
        "min": min(community_sizes),
        "max": max(community_sizes),
        "mean": sum(community_sizes) / len(community_sizes),
        "median": sorted(community_sizes)[len(community_sizes)//2]
    }
    
    print(f"\n📊 THỐNG KÊ KÍCH THƯỚC CỘNG ĐỒNG:")
    print(f"   - Nhỏ nhất: {min(community_sizes)} nodes")
    print(f"   - Lớn nhất: {max(community_sizes)} nodes")
    print(f"   - Trung bình: {sum(community_sizes)/len(community_sizes):.1f} nodes")
    
    return results, communities

# Chạy phân tích Community Detection
community_results, communities = analyze_communities(G, top_k_communities=10)


3. PHÁT HIỆN CỘNG ĐỒNG (COMMUNITY DETECTION)

🔍 Sử dụng thuật toán Louvain...



✓ Phát hiện được 177 cộng đồng
✓ Modularity: 0.5479
   → Modularity > 0.3: Cấu trúc cộng đồng RÕ RÀNG

📊 THỐNG KÊ KÍCH THƯỚC CỘNG ĐỒNG:
   - Nhỏ nhất: 1 nodes
   - Lớn nhất: 198 nodes
   - Trung bình: 9.6 nodes


In [12]:
# Chi tiết top communities
print(f"\n📊 TOP 10 CỘNG ĐỒNG LỚN NHẤT:")
print("-" * 70)

community_results["top_communities"] = []

for i, comm in enumerate(communities[:10], 1):
    # Đếm labels trong community
    label_counts = defaultdict(int)
    for node in comm:
        label = G.nodes[node].get('label', 'Unknown')
        label_counts[label] += 1
    
    # Tìm label chủ đạo
    dominant_label = max(label_counts.items(), key=lambda x: x[1])
    
    # Lấy một số node mẫu
    sample_nodes = list(comm)[:5]
    
    print(f"\n🔹 Cộng đồng {i}: {len(comm)} nodes")
    print(f"   Label chủ đạo: {dominant_label[0]} ({dominant_label[1]} nodes, {100*dominant_label[1]/len(comm):.1f}%)")
    print(f"   Phân bố: {dict(label_counts)}")
    print(f"   Nodes mẫu: {', '.join(sample_nodes)}")
    
    community_results["top_communities"].append({
        "id": i,
        "size": len(comm),
        "dominant_label": dominant_label[0],
        "dominant_label_count": dominant_label[1],
        "dominant_label_percentage": 100 * dominant_label[1] / len(comm),
        "label_distribution": dict(label_counts),
        "sample_nodes": sample_nodes
    })

# Phân tích các cộng đồng đặc biệt
print("\n📊 PHÂN TÍCH CỘNG ĐỒNG:")
print("-" * 70)

# Tìm các cộng đồng có tính chất đặc biệt
artist_communities = []
group_communities = []
mixed_communities = []

for i, comm in enumerate(communities):
    label_counts = defaultdict(int)
    for node in comm:
        label = G.nodes[node].get('label', 'Unknown')
        label_counts[label] += 1
    
    total = len(comm)
    if label_counts.get('Artist', 0) / total > 0.7:
        artist_communities.append((i, len(comm)))
    elif label_counts.get('Group', 0) / total > 0.5:
        group_communities.append((i, len(comm)))
    else:
        mixed_communities.append((i, len(comm)))

print(f"   - Cộng đồng chủ yếu Artist: {len(artist_communities)}")
print(f"   - Cộng đồng chủ yếu Group: {len(group_communities)}")
print(f"   - Cộng đồng hỗn hợp: {len(mixed_communities)}")

community_results["community_types"] = {
    "artist_dominated": len(artist_communities),
    "group_dominated": len(group_communities),
    "mixed": len(mixed_communities)
}

# Kết luận
print("\n" + "-" * 70)
print("📋 KẾT LUẬN VỀ CẤU TRÚC CỘNG ĐỒNG:")
print("-" * 70)

community_sizes = [len(c) for c in communities]
print(f"   1. Mạng K-pop có {len(communities)} cộng đồng rõ ràng")

if community_results.get("modularity", 0) > 0.3:
    print(f"   2. Modularity cao ({community_results.get('modularity', 0):.3f}) cho thấy cấu trúc cộng đồng mạnh")

print(f"   3. Cộng đồng lớn nhất có {max(community_sizes)} nodes ({100*max(community_sizes)/G.number_of_nodes():.1f}% mạng)")

# Diễn giải cộng đồng
print("\n   💡 DIỄN GIẢI:")
print("   - Các cộng đồng có thể đại diện cho:")
print("     + Nghệ sĩ cùng công ty (SM, YG, JYP, HYBE...)")
print("     + Thế hệ idol (1st, 2nd, 3rd, 4th generation)")
print("     + Thể loại âm nhạc (Hip-hop, Ballad, Dance...)")
print("     + Các mối quan hệ hợp tác, collab")


📊 TOP 10 CỘNG ĐỒNG LỚN NHẤT:
----------------------------------------------------------------------

🔹 Cộng đồng 1: 198 nodes
   Label chủ đạo: Artist (97 nodes, 49.0%)
   Phân bố: {'Instrument': 18, 'Group': 12, 'Artist': 97, 'Occupation': 20, 'Company': 30, 'Genre': 17, 'Album': 3, 'Song': 1}
   Nodes mẫu: Instrument_Drums, FT Island, Sandeul, CNU (ca sĩ), After School (nhóm nhạc)

🔹 Cộng đồng 2: 188 nodes
   Label chủ đạo: Artist (107 nodes, 56.9%)
   Phân bố: {'Company': 31, 'Artist': 107, 'Group': 21, 'Occupation': 13, 'Genre': 6, 'Album': 6, 'Instrument': 3, 'Song': 1}
   Nodes mẫu: Company_Mystic Entertainment, Kim Seungmin, Sunday (ca sĩ), Jung Eun-woo, Day6

🔹 Cộng đồng 3: 162 nodes
   Label chủ đạo: Artist (56 nodes, 34.6%)
   Phân bố: {'Artist': 56, 'Genre': 11, 'Group': 34, 'Company': 31, 'Song': 6, 'Occupation': 4, 'Instrument': 3, 'Album': 17}
   Nodes mẫu: Hyomin, Park Sung-hoon, Genre_Electro House, SeeYa, Genre_Electro Swing

🔹 Cộng đồng 4: 119 nodes
   Label chủ đạo:

In [13]:
# Vẽ biểu đồ Community
if MATPLOTLIB_AVAILABLE:
    try:
        import pandas as pd
        
        # Tạo DataFrame cho top communities
        comm_df = pd.DataFrame(community_results["top_communities"])
        
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.barh(range(len(comm_df)), comm_df['size'], color='coral')
        ax.set_yticks(range(len(comm_df)))
        ax.set_yticklabels([f"Cộng đồng {row['id']}: {row['dominant_label']}" for _, row in comm_df.iterrows()], fontsize=9)
        ax.set_xlabel('Số lượng nodes')
        ax.set_title('TOP 10 CỘNG ĐỒNG LỚN NHẤT')
        ax.invert_yaxis()
        plt.tight_layout()
        plt.savefig('community_analysis.png', dpi=150, bbox_inches='tight')
        print("✓ Đã lưu biểu đồ vào community_analysis.png")
        plt.show()
        
        # Hiển thị bảng
        display(comm_df[['id', 'size', 'dominant_label', 'dominant_label_percentage']].head(10))
    except ImportError:
        print("⚠️  Pandas chưa được cài đặt, bỏ qua visualization")

✓ Đã lưu biểu đồ vào community_analysis.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_1572\2260242745.py:19: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


,id,size,dominant_label,dominant_label_percentage
0,1,198,Artist,48.989899
1,2,188,Artist,56.914894
2,3,162,Artist,34.567901
3,4,119,Album,40.336134
4,5,107,Artist,33.644860
5,6,104,Artist,31.730769
6,7,85,Song,42.352941
7,8,80,Song,53.750000
8,9,78,Album,51.282051
9,10,70,Song,32.857143


## 7. Tổng kết

In [14]:
print("\n" + "=" * 70)
print("📊 TỔNG KẾT PHÂN TÍCH MẠNG XÃ HỘI K-POP")
print("=" * 70)

print("\n1️⃣  THẾ GIỚI NHỎ (SMALL WORLD):")
if "is_small_world" in small_world_results:
    sw = small_world_results
    print(f"    - Average Path Length: {sw.get('average_path_length', 'N/A'):.2f}")
    print(f"    - Clustering Coefficient: {sw.get('clustering_coefficient', 'N/A'):.4f}")
    print(f"    - Là Small World: {'✓ CÓ' if sw.get('is_small_world') else '✗ KHÔNG'}")

print("\n2️⃣  PAGERANK - TOP 5 NODES QUAN TRỌNG NHẤT:")
if "top_nodes" in pagerank_results:
    for node_info in pagerank_results.get("top_nodes", [])[:5]:
        print(f"    {node_info['rank']}. {node_info['node']} ({node_info['label']}) - PageRank: {node_info['pagerank']:.6f}")

print("\n3️⃣  CỘNG ĐỒNG:")
if "total_communities" in community_results:
    comm = community_results
    print(f"    - Số cộng đồng: {comm.get('total_communities', 'N/A')}")
    if comm.get('modularity'):
        print(f"    - Modularity: {comm.get('modularity', 'N/A'):.4f}")

print("\n" + "=" * 70)
print("✓ HOÀN TẤT PHÂN TÍCH")
print("=" * 70)


📊 TỔNG KẾT PHÂN TÍCH MẠNG XÃ HỘI K-POP

1️⃣  THẾ GIỚI NHỎ (SMALL WORLD):
    - Average Path Length: 3.94
    - Clustering Coefficient: 0.1124
    - Là Small World: ✓ CÓ

2️⃣  PAGERANK - TOP 5 NODES QUAN TRỌNG NHẤT:
    1. Occupation_Diễn viên (Occupation) - PageRank: 0.023980
    2. Genre_R&B (Genre) - PageRank: 0.020009
    3. Genre_Dance-pop (Genre) - PageRank: 0.012662
    4. Genre_Hip hop (Genre) - PageRank: 0.012055
    5. Occupation_Nhạc sĩ (Occupation) - PageRank: 0.010494

3️⃣  CỘNG ĐỒNG:
    - Số cộng đồng: 177
    - Modularity: 0.5479

✓ HOÀN TẤT PHÂN TÍCH
